In [1]:
import re
import os
import sys
import math
import json
import shutil
import hashlib
import platform
import itertools
import collections
import pkg_resources  # pip install py-rouge
from io import open
from rouge import Rouge
from konlpy.tag import Mecab 

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm
from zipfile import ZipFile

import boto3
from botocore import UNSIGNED
from botocore.client import Config

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import _LRScheduler
import transformers
from transformers import get_scheduler, PreTrainedTokenizerFast, EarlyStoppingCallback, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

C:\Users\hist\miniconda3\envs\py38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Trainer arguments
lr = 1e-3
stop = 3
epoch = 100
batch = 4
seed = 42
device = 'cuda'

# init

In [3]:
class AwsS3Downloader(object):
    def __init__(
        self,
        aws_access_key_id=None,
        aws_secret_access_key=None,
    ):
        self.resource = boto3.Session(
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        ).resource("s3")
        self.client = boto3.client(
            "s3",
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            config=Config(signature_version=UNSIGNED),
        )

    def __split_url(self, url: str):
        if url.startswith("s3://"):
            url = url.replace("s3://", "")
        bucket, key = url.split("/", maxsplit=1)
        return bucket, key

    def download(self, url: str, local_dir: str):
        bucket, key = self.__split_url(url)
        filename = os.path.basename(key)
        file_path = os.path.join(local_dir, filename)

        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        meta_data = self.client.head_object(Bucket=bucket, Key=key)
        total_length = int(meta_data.get("ContentLength", 0))

        downloaded = 0

        def progress(chunk):
            nonlocal downloaded
            downloaded += chunk
            done = int(50 * downloaded / total_length)
            sys.stdout.write(
                "\r{}[{}{}]".format(file_path, "█" * done, "." * (50 - done))
            )
            sys.stdout.flush()

        try:
            with open(file_path, "wb") as f:
                self.client.download_fileobj(bucket, key, f, Callback=progress)
            sys.stdout.write("\n")
            sys.stdout.flush()
        except:
            raise Exception(f"downloading file is failed. {url}")
        return file_path

def download(url, chksum=None, cachedir=".cache"):
    cachedir_full = os.path.join(os.getcwd(), cachedir)
    os.makedirs(cachedir_full, exist_ok=True)
    filename = os.path.basename(url)
    file_path = os.path.join(cachedir_full, filename)
    if os.path.isfile(file_path):
        if hashlib.md5(open(file_path, "rb").read()).hexdigest()[:10] == chksum:
            print(f"using cached model. {file_path}")
            return file_path, True

    s3 = AwsS3Downloader()
    file_path = s3.download(url, cachedir_full)
    if chksum:
        assert (
            chksum == hashlib.md5(open(file_path, "rb").read()).hexdigest()[:10]
        ), "corrupted file!"
    return file_path, False

def get_kobart_tokenizer(cachedir=".cache"):
    """Get KoGPT2 Tokenizer file path after downloading"""
    tokenizer = {
        "url": "s3://skt-lsl-nlp-model/KoBART/tokenizers/kobart_base_tokenizer_cased_cf74400bce.zip",
        "chksum": "cf74400bce",
    }
    file_path, is_cached = download(
        tokenizer["url"], tokenizer["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.expanduser(cachedir)
    if (
        not os.path.exists(os.path.join(cachedir_full, "emji_tokenizer"))
        or not is_cached
    ):
        if not is_cached:
            shutil.rmtree(
                os.path.join(cachedir_full, "emji_tokenizer"), ignore_errors=True
            )
        zipf = ZipFile(os.path.expanduser(file_path))
        zipf.extractall(path=cachedir_full)
    tok_path = os.path.join(cachedir_full, "emji_tokenizer/model.json")
    tokenizer_obj = PreTrainedTokenizerFast(
        tokenizer_file=tok_path,
        bos_token="<s>",
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        mask_token="<mask>",
    )
    return tokenizer_obj

def get_pytorch_kobart_model(ctx="cpu", cachedir=".cache"):
    pytorch_kobart = {
        "url": "s3://skt-lsl-nlp-model/KoBART/models/kobart_base_cased_ff4bda5738.zip",
        "chksum": "ff4bda5738",
    }
    model_zip, is_cached = download(
        pytorch_kobart["url"], pytorch_kobart["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.join(os.getcwd(), cachedir)
    model_path = os.path.join(cachedir_full, "kobart_from_pretrained")
    if not os.path.exists(model_path) or not is_cached:
        if not is_cached:
            shutil.rmtree(model_path, ignore_errors=True)
        zipf = ZipFile(os.path.expanduser(model_zip))
        zipf.extractall(path=cachedir_full)
    return model_path

In [4]:
def make_df(phase):
    work_dir = "C:\\Users\\hist\\Documents\\GitHub\\KoBART"
    if phase == 'train':
        tmp = work_dir+'/022.요약문 및 레포트 생성 데이터/01.데이터/1.Training/라벨링데이터/TL1'
    else:
        tmp = work_dir+'/022.요약문 및 레포트 생성 데이터/01.데이터/2.Validation/라벨링데이터/VL1'
    listdir = os.listdir(tmp)
    df = pd.DataFrame({}, columns = ['genre', 'text', 'label'])
    for i in listdir:
        files = os.listdir(f'{tmp}/{i}/2~3sent')
        for f in tqdm(files):
            with open(f'{tmp}/{i}/2~3sent/{f}', 'r', encoding='utf-8') as json_file:
                j = json.loads(json_file.read())
                df2 = pd.DataFrame.from_dict([{'genre' : i, 
                                               'text'  : j['Meta(Refine)']['passage'], 
                                               'label' : j['Annotation']['summary1']}])
                df = pd.concat([df, df2])
    return df

In [5]:
# %%time

# train = make_df('train').reset_index(drop=True)
# val = make_df('val').reset_index(drop=True)

In [6]:
# train.to_parquet('train.parquet')
# val.to_parquet('val.parquet')

In [7]:
train = pd.read_parquet('train.parquet')
val   = pd.read_parquet('val.parquet')
val   = val.sample(n=2, replace=False).reset_index(drop=True)
val

,genre,text,label
0,09.literature,"우징이, 염장, 정년(閻長,鄭年) 등 여섯 장군으로 하여금 군사를 인솔하고 북를 두...",점령 지대를 넓히며 동진한 이 군대는 신라 대감 김민주 등 여러 곳에서 그곳 수장에...
1,03.his_cul,"태조산에 있는 절로, 고려 태조(재위 918~943) 때 도선국사가 처음 세웠다는 ...",설화에 따르면 백학 한 쌍이 대웅전 뒤쪽 암벽에 불상을 조각하고 날아간 자리에 도선...


# 전처리

In [8]:
def preprocss(df):
    df.text = df.text.apply(lambda x : re.sub('\n', ' ',  x))
    df.text = df.text.apply(lambda x : re.sub(' +', ' ',  x).strip())
    return df

train = preprocss(train)
val = preprocss(val)

In [9]:
val.loc[0, 'text']

'우징이, 염장, 정년(閻長,鄭年) 등 여섯 장군으로 하여금 군사를 인솔하고 북를 두드리며 무주성에 들 때 같은 때는 군용이 진실로 당당하고 성하였 다. 신왕인 김명에게는 반역군이요 국가적으로는 토역(討逆)군인 이 군대는, 착일착 점령지대를 넓히며 동진하였다. 여러 곳에서 그곳 수장(守將)에게 반항를 받았지만, 반항하는 자는 모두 참패하였다. 신라대감(新羅大監) 김민주(金敏周) 같은 사람은 적지 않은 군사를 끌어가 지고 반항를 하였지만 우징의 막하 장군 낙금, 이순행(駱金,李順行) 등이 마병(馬兵)으로써 돌격를 하여 이를 평정하였다. 토역군인지 반역군인지 장 차 결과를 보아야 밝혀질 우징의 군대는, 평동장군 김양의 지휘 아래, 옛날 의 국경도 무사히 넘어, 동진(東進)를 계속하여 새해 정월 열아흐렛날은 대 구(大丘)에 이르렀다. 인제는 서울도 지호 간이었다. 요 며칠 전에 선왕를 목매게 하고 스스로 서서 임금이 된 김명은, 관군(官 軍)를 호령하여 나아가 맞아 싸우게 하였다. 관군를 내어 보내기는 하고도 왕은 스스로 절망의 탄성를 연하여 내었다. 위에 오른 지 불과 수삼 삭, 아 직 내 백성 내 군사라고 믿를 만한 사람은 얻지 못하였는데, 돌연한 이 변 란이요, 더우기 변란의 주인인 우징은 그의 아버지의대부터 관민간에 많은 존경과 애모를 받던 사람이었다. 이러한 입장이매 분명한 패배요 멸망이다. 근신 시종들도 모두 도망가고, 겨우 붙들어 둔 두세 명를 데리고, 왕은 망 루(望樓)에서 형편 형세를 살피고 있었다. 대구를 벌써 지난 적군(?)이매, 관군과는 서울 근교에서 부딪칠 것이다. 부딪치면 그 결과는? 분명한 패배일 줄 알면서도, 그래도 천에 하나의 요행를 기다리는 왕은 시 종들과 교외 쪽를 바라보고 있었다.'

In [10]:
val.loc[0, 'label']

'점령 지대를 넓히며 동진한 이 군대는 신라 대감 김민주 등 여러 곳에서 그곳 수장에게 반항를 받았지만 반항하는 자는 모두 참패하였다.'

In [11]:
train.text.str.len().min()

242

# Dataset

In [12]:
class KoBARTSumDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, ignore_index=-100):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.df = df
        self.len = len(self.df)

        self.pad_index = self.tokenizer.pad_token_id
        self.ignore_index = ignore_index

    def add_padding_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs

    def add_ignored_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.ignore_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs
    
    def __getitem__(self, idx):
        instance = self.df.iloc[idx]
        input_ids = self.tokenizer.encode(instance['text'])
        input_ids = self.add_padding_data(input_ids)

        label_ids = self.tokenizer.encode(instance['label'])
        label_ids.append(self.tokenizer.eos_token_id)
        dec_input_ids = [self.tokenizer.eos_token_id]
        dec_input_ids += label_ids[:-1]
        dec_input_ids = self.add_padding_data(dec_input_ids)
        label_ids = self.add_ignored_data(label_ids)
        
        return {'input_ids': np.array(input_ids, dtype=np.int_),
                'decoder_input_ids': np.array(dec_input_ids, dtype=np.int_),
                'labels': np.array(label_ids, dtype=np.int_)}

    def __len__(self):
        return self.len
train_dataset = KoBARTSumDataset(train, PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1'), 1024)
val_dataset = KoBARTSumDataset(val, PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1'), 1024)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [13]:
train.text.str.len().max(), val.text.str.len().max(), 

(1499, 857)

# Model

In [14]:
model = AutoModelForSeq2SeqLM.from_pretrained('gogamza/kobart-base-v1').to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [15]:
collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=tokenizer.pad_token_id)

In [16]:
rouge = Rouge()
def compute_metrics(pred):
    preds, labels = pred

    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    print('Text\n', val.text[0])
    print(f'Summarize\nGold : {labels[0]}\nGen : {preds[0]}')
    
    labels = ['\n'.join(labels)]
    preds = ['\n'.join(preds)]
    score = rouge.get_scores(preds, labels, avg=True)
    return {
        "ROUGE-1" : score['rouge-1'],
        "ROUGE-2" : score['rouge-2'],
        "ROUGE-L" : score['rouge-l'],
    }

In [17]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=1, eta_min=0, last_epoch=-1)

lr_scheduler = transformers.get_linear_schedule_with_warmup(optimizer=optimizer, 
                                                            num_warmup_steps=100, 
                                                            num_training_steps=epoch * len(train_dataset) * batch, 
                                                            last_epoch = -1)

In [ ]:
args = Seq2SeqTrainingArguments(run_name = f'KoBARTSum',
                                output_dir= f"models",
                                evaluation_strategy="steps",
                                eval_steps=10,
                                save_steps=10,
                                logging_steps=10,
                                save_total_limit = 2,
                                
                                per_device_train_batch_size=batch,
                                per_device_eval_batch_size=batch,
                                gradient_accumulation_steps=16,
                                num_train_epochs=epoch,
                                
                                load_best_model_at_end=True,
                                fp16=True,
                                do_train=True,
                                do_eval=True,
                                predict_with_generate=True,)

trainer = Seq2SeqTrainer(model=model,
                         tokenizer=tokenizer,
                         args=args,

                         train_dataset=train_dataset,
                         eval_dataset=val_dataset,

                         compute_metrics=compute_metrics,
                         optimizers=(optimizer, lr_scheduler),
                         callbacks=[EarlyStoppingCallback(early_stopping_patience=stop)],
                         data_collator=collator,)
trainer.train()

Using cuda_amp half precision backend
***** Running training *****
  Num examples = 73340
  Num Epochs = 100
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 114500
  Number of trainable parameters = 123859968
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
